In [1]:
from IPython.display import clear_output
from pathlib import Path
import pandas as pd
import plotly.express as px
import plotly.io as pio
import requests
import json
import ast

In [ ]:
# Load the main dataset
df = pd.read_csv(
    Path("../../../../../data/processed_data/ev_main_dataset.csv"),
)

# Conver the `ev_adoption_rate` and `ev_growth_rate` to percentages
df["ev_adoption_rate"] = df["ev_adoption_rate"] * 100
df["ev_growth_rate"] = df["ev_growth_rate"] * 100

# Display the data
print("Shape:", df.shape)
df.head()

In [ ]:
# Create a seperate DataFrame for Florida
tx_df = df[df["state"] == "TX"].sort_values(["year", "state", "county"]).reset_index(drop=True)

# Display the DataFrames
display(tx_df.head(2))

In [ ]:
ex_tx_df = tx_df.copy()

# Filter for the latest year values
ex_tx_df = ex_tx_df.loc[ex_tx_df["year"] == 2022]

# Inspect the 'zip_codes' column class
print("Before:", type(ex_tx_df["zip_codes"].values[0]))

# Convert the strings in the 'zip_codes' column back into lists
ex_tx_df["zip_codes"] = ex_tx_df["zip_codes"].apply(ast.literal_eval)

# Confirm the conversion from string to list for the 'zip_codes' column
print("After:", type(ex_tx_df["zip_codes"].values[0]))
ex_tx_df.head()

In [ ]:
# Explode the 'zip_codes' column so that each zip code has its own row
ex_tx_df = ex_tx_df.explode("zip_codes")

# Check the `zip_codes` column values data type
print(type(ex_tx_df["zip_codes"].values[0]))
ex_tx_df.head()

In [ ]:
# Convert again to string since it is now an integer after exploding
ex_tx_df["zip_codes"] = ex_tx_df["zip_codes"].astype(str).str.strip()

# Confirm the conversion from integer to string for the 'zip_codes' column
print(type(ex_tx_df["zip_codes"].values[0]))
ex_tx_df.head()

In [ ]:
# Obtain the geojson data for Texas
state_geojson_url = "https://raw.githubusercontent.com/OpenDataDE/State-zip-code-GeoJSON/master/tx_texas_zip_codes_geo.min.json"
texas_geojson = requests.get(state_geojson_url).json()

# Inspect the geojson data
print(json.dumps(texas_geojson, indent=4))

In [8]:
# Use `iframe` to display the plot in JupyterLab

# pio.renderers.default = 'notebook'
pio.renderers.default = 'iframe' 

In [9]:
# Use plotly express to create a choropleth map and display it
# The zip_code' field in the DataFrame matches the field in the GeoJSON "ZCTA5CE10" 
# The 'featureidkey' parameter is used to link the GeoJSON to the DataFrame
# Documentation: https://plotly.github.io/plotly.py-docs/generated/plotly.express.choropleth_mapbox.html

fig = px.choropleth_mapbox(ex_tx_df,
                           geojson=texas_geojson,
                           featureidkey="properties.ZCTA5CE10",
                           locations='zip_codes',
                           color='ev_adoption_rate',
                           hover_name='county',
                           hover_data=['ev_registrations', 'median_income'],
                           labels={
                               'county': 'County',
                               'zip_codes': 'Zip Code',
                               'ev_adoption_rate': 'EV Adoption Rate (%)',
                               'ev_registrations': 'EV Registrations',
                               'median_income': 'Median Income'
                           },
                           color_continuous_scale='Viridis',
                           opacity=0.8,
                           zoom=5,
                           center={"lat": 27.9944024, "lon": -81.7602544},
                           mapbox_style="carto-positron",
                           title="Florida EV Adoption Rate by County",
                           )

fig.update_layout(
    coloraxis_colorbar_title="EV Adoption Rate (%)",
    margin={"r": 0, "t": 0, "l": 0, "b": 0}
)

file_name = "tx_geo_map_ev_adoption_rate.html"
file_path = Path(f"../../../../../reports/geospatial_maps/{file_name}")
fig.write_html(file_path)

clear_output()